In [ ]:
import hydra
import torch
from accelerate import PartialState
from hydra import compose, initialize
from omegaconf import OmegaConf

PartialState()

with initialize(version_base=None, config_path="../configs", job_name="stats_db_ipynb"):
    cfg = compose(config_name="train", overrides=["experiment=tdda_3d_baseline"])

cfg.datamodule._training_calvin_data = "../" + cfg.datamodule._training_calvin_data  # noqa: SLF001
cfg.datamodule._instructions = "../" + cfg.datamodule._instructions  # noqa: SLF001

ds_cfg = cfg.datamodule.datamodule.train_dataloader.dataset
print(OmegaConf.to_yaml(ds_cfg))

In [ ]:
ds = hydra.utils.instantiate(ds_cfg)
ds

In [ ]:
from tqdm.auto import tqdm

ds_len = len(ds)
stat = torch.zeros(2, ds_len, ds[0]["trajectory"].shape[-1])
for i in tqdm(range(ds_len)):
    tj = ds[i]["trajectory"]
    stat[0, i] = ds[i]["trajectory"].amin(dim=[0, 1])
    stat[1, i] = ds[i]["trajectory"].amax(dim=[0, 1])
stats = torch.zeros(2, ds[0]["trajectory"].shape[-1])
stats[0] = stat[0].amin(dim=0)
stats[1] = stat[1].amax(dim=0)

stats

In [ ]:
minn = stats.tolist()
t_minn = torch.tensor(minn)

tt = torch.ones(2, 3)
tt[:, :] = t_minn[:, :3]

tt